In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
X_full = pd.read_csv('./predict-supercars-prices-2025/supercars_train.csv', index_col='id')
X_test_full = pd.read_csv('./predict-supercars-prices-2025/supercars_test.csv', index_col='id')

In [3]:
X_full.head()

,year,brand,color,carbon_fiber_body,engine_config,horsepower,torque,weight_kg,zero_to_60_s,top_speed_mph,...,has_warranty,last_service_date,service_history,non_original_parts,model,warranty_years,damage,damage_cost,damage_type,price
id,,,,,,,,,,,,,,,,,,,,,
wn8zA4ADUC,2023,McLaren,Silver,1,Hybrid,1045,794,1897,3.63,227,...,0,2024-08-15,authorized,0,600LT,0,1,83632.0,major,329510.72
m5EyycSRrS,2024,Aston Martin,Black,1,W16,879,510,2193,2.72,247,...,0,2025-03-29,none,1,Valhalla,0,0,NaN,NaN,509289.91
IOcII96Ua3,2022,Koenigsegg,White,1,V12,609,489,1277,3.16,236,...,0,2025-06-30,authorized,0,Jesko,0,1,51179.0,major,2164428.25
qlds6yyR3r,2024,Bugatti,Blue,0,V12,1044,1162,1733,2.15,248,...,1,2024-08-29,none,0,Chiron,1,0,NaN,NaN,2793655.40
fRR6ulIWvU,2024,Pagani,White,0,V12,853,679,1307,2.35,247,...,0,2025-02-21,authorized,0,Zonda,0,1,77100.0,major,2927390.00


In [4]:
X_test_full.head()

,year,brand,color,carbon_fiber_body,engine_config,horsepower,torque,weight_kg,zero_to_60_s,top_speed_mph,...,limited_edition,has_warranty,last_service_date,service_history,non_original_parts,model,warranty_years,damage,damage_cost,damage_type
id,,,,,,,,,,,,,,,,,,,,,
N4C4A2ICG2,2025,Aston Martin,Yellow,0,V12,1037,720,2285,2.94,247,...,0,1,2024-07-23,independent,0,Valhalla,5,1,92320.0,minor
aIB01tNqkz,2024,McLaren,Black,0,V10,831,490,1286,2.37,262,...,0,1,2024-10-29,authorized,0,600LT,3,0,NaN,NaN
0AX3BsniiV,2025,Bugatti,White,1,Electric,792,714,1810,3.09,244,...,0,0,2024-10-07,independent,0,Veyron,0,0,NaN,NaN
3J6iupWk0z,2021,Bugatti,Silver,0,V12,1229,511,2347,2.64,205,...,0,0,2025-05-18,authorized,0,Veyron,0,0,NaN,NaN
7atXIEFcBF,2023,Lamborghini,Black,1,Electric,535,512,1845,3.86,244,...,0,1,2024-12-03,authorized,0,Huracan,3,1,67822.0,totaled


In [5]:
X_full.dropna(axis=0, subset='price', inplace=True)
y = X_full['price']
X_full.drop(['price'], axis=1, inplace=True)

In [6]:
X_full.columns

Index(['year', 'brand', 'color', 'carbon_fiber_body', 'engine_config',
       'horsepower', 'torque', 'weight_kg', 'zero_to_60_s', 'top_speed_mph',
       'num_doors', 'transmission', 'drivetrain', 'market_region', 'mileage',
       'num_owners', 'interior_material', 'brake_type', 'tire_brand',
       'aero_package', 'limited_edition', 'has_warranty', 'last_service_date',
       'service_history', 'non_original_parts', 'model', 'warranty_years',
       'damage', 'damage_cost', 'damage_type'],
      dtype='object')

In [7]:
X_full.isnull().sum()

year                     0
brand                    0
color                    0
carbon_fiber_body        0
engine_config            0
horsepower               0
torque                   0
weight_kg                0
zero_to_60_s             0
top_speed_mph            0
num_doors                0
transmission             0
drivetrain               0
market_region            0
mileage                  0
num_owners               0
interior_material        0
brake_type               0
tire_brand               0
aero_package             0
limited_edition          0
has_warranty             0
last_service_date        0
service_history          0
non_original_parts       0
model                    0
warranty_years           0
damage                   0
damage_cost           1402
damage_type           1402
dtype: int64

In [8]:
X_train,X_valid,y_train,y_valid = train_test_split(X_full,y,train_size=0.8,random_state=0)

In [9]:
categorical_cols = [cols for cols in X_train.columns
                    if X_train[cols].dtype == 'object'
                    and X_train[cols].nunique() < 10]
numerical = [cols for cols in X_train.columns
             if X_train[cols].dtype in ['int64','float64']]

In [10]:
print("categorical columns is ", categorical_cols)
print(len(categorical_cols))
print("numerical cols is a ", numerical)
print(len(numerical))

categorical columns is  ['brand', 'color', 'engine_config', 'transmission', 'drivetrain', 'market_region', 'interior_material', 'brake_type', 'tire_brand', 'service_history', 'damage_type']
11
numerical cols is a  ['year', 'carbon_fiber_body', 'horsepower', 'torque', 'weight_kg', 'zero_to_60_s', 'top_speed_mph', 'num_doors', 'mileage', 'num_owners', 'aero_package', 'limited_edition', 'has_warranty', 'non_original_parts', 'warranty_years', 'damage', 'damage_cost']
17


In [11]:
my_cols = numerical + categorical_cols
print(my_cols)
print(len(my_cols))

['year', 'carbon_fiber_body', 'horsepower', 'torque', 'weight_kg', 'zero_to_60_s', 'top_speed_mph', 'num_doors', 'mileage', 'num_owners', 'aero_package', 'limited_edition', 'has_warranty', 'non_original_parts', 'warranty_years', 'damage', 'damage_cost', 'brand', 'color', 'engine_config', 'transmission', 'drivetrain', 'market_region', 'interior_material', 'brake_type', 'tire_brand', 'service_history', 'damage_type']
28


In [12]:
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()
X_test_full = X_test_full[my_cols].copy()

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [14]:
numerical_tra = SimpleImputer(strategy= 'constant')
categorical_tra = Pipeline(
    steps =[
        ('imputer', SimpleImputer(strategy='constant')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)

In [15]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_tra,numerical),
        ('cat', categorical_tra,categorical_cols)
    ]
)

In [16]:
model = RandomForestRegressor(n_estimators=1500, max_depth=5, random_state=0, n_jobs=-1)

In [17]:
clf = Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',model)
])

In [18]:
clf.fit(X_train,y_train)
pred = clf.predict(X_valid)
print("MAE : ",mean_absolute_error(y_valid,pred))

MAE :  304365.1253857289


In [19]:
model1 = RandomForestRegressor(n_estimators=2000, max_depth=5, random_state=0, n_jobs=-1)
clf = Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',model1)
])
clf.fit(X_train,y_train)
pred = clf.predict(X_valid)
print("MAE : ",mean_absolute_error(y_valid,pred))

MAE :  304323.770892214


In [20]:
preds = clf.predict(X_test_full)
submission = pd.DataFrame({'id': X_test_full.index, 'target': preds})
submission.to_csv('submission.csv', index=False)